<h1 style='text-align: center'>Raifhack-DS-2021-Fall 👽</h1>

<p  style='text-align: center'>
This notebook is in <span style='color: green; font-weight: 700'>Active</span> state of development! Check out this notebook to see some updates as I update new stuff as oftern as I learn it!
<a style='font-weight:700' href='https://github.com/LilDataScientist/Raifhack-DS-2021-Fall'> Code on GitHub! </a></p>

<div style='text-align: center'>
    <img src='https://i.postimg.cc/HLF8SsJK/B041-Bank-of-the-West.jpg' width='700' />
</div>

In [1]:
import numpy as np 
import pandas as pd

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

# Models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Pipelines
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

# Helpers
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error, make_scorer, mean_absolute_error


sns.set_theme()

In [2]:
df_train = pd.read_csv('../input/raifhackds2021fall/data/train.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Quick view</h1>

In [3]:
df_train

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
0,Пермь,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,...,5.762963,5.530612,1964.118519,1960.959184,Пермский край,32.000000,S27289,2020-01-05,10,0
1,Шатура,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,...,2.894366,3.527778,1952.321678,1957.222222,Московская область,280.000000,S17052,2020-01-05,10,0
2,Ярославль,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,...,6.141414,7.222222,1968.150000,1973.370370,Ярославская область,297.400000,S16913,2020-01-05,110,0
3,Новокузнецк,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,...,8.581081,9.000000,1992.716216,2014.000000,Кемеровская область,190.000000,S10148,2020-01-05,110,0
4,Москва,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,...,7.263889,5.684211,1963.229167,1960.500000,Москва,60.200000,S1338,2020-01-05,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279787,Томск,1.0,COL_280111,56.459183,84.979334,2,33,111,222,0,...,5.714286,5.882353,1972.260870,1973.460000,Томская область,358.320073,S11114,2020-08-23,10,1
279788,Санкт-Петербург,1.0,COL_280479,59.936954,30.356383,10,274,718,1340,0,...,4.719388,4.706667,1876.994898,1873.186667,Санкт-Петербург,119.637556,S28440,2020-08-23,110,1
279789,Калининград,3.0,COL_280518,54.729233,20.514968,0,12,34,84,0,...,3.950413,4.885714,1964.258333,1970.571429,Калининградская область,312.789725,S6671,2020-08-23,10,1
279790,Кемерово,1.0,COL_280529,55.360680,86.081460,5,57,100,134,0,...,4.691489,4.125000,1957.425532,1954.625000,Кемеровская область,89.201305,S17667,2020-08-23,110,1


<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Describe data</h1>

In [4]:
df_train.describe()

,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,...,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,total_square,realty_type,price_type
count,279792.000000,279792.000000,279792.000000,279792.000000,279792.000000,279792.000000,279792.000000,279792.000000,279792.000000,279792.000000,...,279792.000000,265196.000000,252558.000000,263084.000000,249624.000000,263553.000000,250155.000000,279792.000000,279792.000000,279792.000000
mean,54.364078,47.763540,2.709084,40.605146,81.596171,133.285458,0.037442,0.885701,2.046467,3.748163,...,30.110661,2042.541716,644.610557,7.051233,7.360464,1967.532599,1967.988580,507.833604,54.974088,0.016058
std,4.245713,17.044625,4.202451,53.293388,105.193169,172.290136,0.391014,6.858338,14.801566,25.679859,...,27.686234,1359.884747,445.699329,3.542084,4.231369,45.807699,54.110015,1704.251771,47.856417,0.125700
min,42.651897,19.892178,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,5.100000,10.000000,0.000000
25%,53.226600,37.582988,0.000000,7.000000,16.000000,28.000000,0.000000,0.000000,0.000000,0.000000,...,10.000000,932.000000,290.000000,4.591837,4.619959,1960.070000,1959.890097,65.900000,10.000000,0.000000
50%,55.679090,39.702435,1.000000,22.000000,46.000000,77.000000,0.000000,0.000000,0.000000,0.000000,...,25.000000,1949.000000,602.000000,6.368932,6.395349,1970.890411,1971.647059,128.737034,10.000000,0.000000
75%,56.306976,55.957523,4.000000,51.000000,101.000000,164.000000,0.000000,0.000000,1.000000,2.000000,...,43.000000,2978.000000,936.000000,8.698925,9.100000,1983.701754,1986.950000,336.000000,110.000000,0.000000
max,69.500740,151.777000,46.000000,468.000000,851.000000,1392.000000,30.000000,586.000000,949.000000,1162.000000,...,289.000000,18392.000000,6105.000000,53.717949,221.666667,2019.000000,2020.000000,40000.000000,110.000000,1.000000


<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Missing values</h1>

In [5]:
def print_missing_values(df):
    missing_values_df = pd.DataFrame(columns=['dtype','Feature', 'Number Of Missing Values', 'Percentage of Missing values'])

    for i in df:
        if df[i].isnull().sum() != 0:
            dtype = df[i].dtype
            feature                      = i
            number_of_missing_values     = df[i].isnull().sum()
            percentage_of_missing_values = round(number_of_missing_values / len(df.index) * 100, 2)
            missing_values_df = missing_values_df.append({
                                      'dtype': dtype,
                                      'Feature': i,
                                      'Number Of Missing Values': number_of_missing_values,
                                      'Percentage of Missing values': percentage_of_missing_values
            }, ignore_index=True)

    return missing_values_df

In [6]:
print_missing_values(df_train)

,dtype,Feature,Number Of Missing Values,Percentage of Missing values
0,object,floor,176237,62.99
1,float64,osm_city_nearest_population,55,0.02
2,float64,reform_house_population_1000,14596,5.22
3,float64,reform_house_population_500,27234,9.73
4,float64,reform_mean_floor_count_1000,16708,5.97
5,float64,reform_mean_floor_count_500,30168,10.78
6,float64,reform_mean_year_building_1000,16239,5.80
7,float64,reform_mean_year_building_500,29637,10.59
8,object,street,1606,0.57


<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Unique values</h1>

In [7]:
def print_unique_values(df):
    unique_values_df = pd.DataFrame(columns=['Feature', 'Number of values', 'Values'])

    for col in df:
        unique_values_df = unique_values_df.append({
                                      'Feature': col,
                                      'Number of values': df[col].unique().shape[0],
                                      'Values': df[col].unique()
            }, ignore_index=True)
    
    pd.set_option('display.max_rows', len(unique_values_df))
    return unique_values_df

In [8]:
print_unique_values(df_train)

,Feature,Number of values,Values
0,city,4948,"[Пермь, Шатура, Ярославль, Новокузнецк, Москва..."
1,floor,207,"[nan, 3.0, 4.0, -1.0, 1.0, 14.0, 2.0, 8.0, -2...."
2,id,279792,"[COL_0, COL_1, COL_2, COL_3, COL_4, COL_5, COL..."
3,lat,126190,"[57.998207, 55.574284, 57.61914, 53.897083, 55..."
4,lng,131884,"[56.292797, 39.543835, 39.850525, 87.108604, 3..."
5,osm_amenity_points_in_0.001,46,"[4, 3, 1, 0, 5, 10, 2, 7, 6, 19, 9, 11, 8, 15,..."
6,osm_amenity_points_in_0.005,412,"[19, 24, 30, 0, 23, 18, 38, 92, 66, 12, 5, 10,..."
7,osm_amenity_points_in_0.0075,804,"[35, 37, 67, 5, 64, 38, 0, 117, 256, 138, 32, ..."
8,osm_amenity_points_in_0.01,1273,"[52, 59, 128, 21, 153, 50, 0, 219, 376, 249, 5..."
9,osm_building_points_in_0.001,25,"[0, 2, 3, 1, 9, 6, 5, 10, 16, 4, 13, 7, 14, 11..."


In [9]:
ID = df_train['id']
df_train.drop(['id', 'lat', 'lng', 'osm_city_nearest_name', 'date', 'street', 'floor', 'osm_city_closest_dist'], axis=1, inplace=True)

In [10]:
print_unique_values(df_train)

,Feature,Number of values,Values
0,city,4948,"[Пермь, Шатура, Ярославль, Новокузнецк, Москва..."
1,osm_amenity_points_in_0.001,46,"[4, 3, 1, 0, 5, 10, 2, 7, 6, 19, 9, 11, 8, 15,..."
2,osm_amenity_points_in_0.005,412,"[19, 24, 30, 0, 23, 18, 38, 92, 66, 12, 5, 10,..."
3,osm_amenity_points_in_0.0075,804,"[35, 37, 67, 5, 64, 38, 0, 117, 256, 138, 32, ..."
4,osm_amenity_points_in_0.01,1273,"[52, 59, 128, 21, 153, 50, 0, 219, 376, 249, 5..."
5,osm_building_points_in_0.001,25,"[0, 2, 3, 1, 9, 6, 5, 10, 16, 4, 13, 7, 14, 11..."
6,osm_building_points_in_0.005,195,"[0, 1, 4, 20, 8, 2, 7, 12, 27, 3, 5, 6, 11, 10..."
7,osm_building_points_in_0.0075,336,"[0, 1, 13, 2, 29, 14, 4, 10, 6, 8, 17, 15, 57,..."
8,osm_building_points_in_0.01,469,"[0, 1, 26, 4, 37, 3, 23, 2, 21, 13, 8, 11, 7, ..."
9,osm_catering_points_in_0.001,24,"[0, 1, 2, 4, 5, 3, 6, 10, 12, 8, 7, 9, 11, 17,..."


<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Location [Feature engineering]</h1>

In [11]:
def location(df):
    is_moscow = []
    is_moscow_oblast = []
    is_region = []

    for row in df['region']:
        if row == 'Москва':
            is_moscow.append(1)
            is_moscow_oblast.append(0)
            is_region.append(0)
        elif row == 'Московская область':
            is_moscow.append(0)
            is_moscow_oblast.append(1)
            is_region.append(0)
        else:
            is_moscow.append(0)
            is_moscow_oblast.append(0)
            is_region.append(1)

    df['is_moscow'] = is_moscow
    df['is_moscow_oblast'] = is_moscow_oblast
    df['is_region'] = is_region

    df.drop(['region'], axis=1, inplace=True)
    
    print('Moscow: ' + str((len(df[df['is_moscow'] == 1]))))
    print('Moscow oblast: ' + str((len(df[df['is_moscow_oblast'] == 1]))))
    print('Region: ' + str((len(df[df['is_region'] == 1]))))

location(df_train)

Moscow: 55962
Moscow oblast: 16328
Region: 207502


<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>CIies with > 1M people [Feature engineering]</h1>

In [12]:
def cities_1m(df):
    cities_1m_people = ['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Самара', 'Омск',
                       'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Воронеж', 'Пермь', 'Волгоград']

    is_million = []
    is_not_million = []

    for row in df['city']:
        if row in cities_1m_people:
            is_million.append(1)
            is_not_million.append(0)
        else:
            is_million.append(0)
            is_not_million.append(1)

    df['is_million'] = is_million
    df['is_not_million'] = is_not_million

    df.drop(['city'], axis=1, inplace=True)
    
    print('is_million: ' + str((len(df[df['is_million'] == 1]))))
    print('is_not_million: ' + str((len(df[df['is_not_million'] == 1]))))
    
cities_1m(df_train)

is_million: 139023
is_not_million: 140769


<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>District rate [Feature engineering]</h1>

In [13]:
def make_district(df):
    district_rate = []

    for row in df['realty_type']:
        if row == 110:
            district_rate.append(1)
        elif row == 100:
            district_rate.append(10)
        elif row == 10:
            district_rate.append(0)

    df['district_rate'] = district_rate

    df.drop(['realty_type'], axis=1, inplace=True)
    
make_district(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Offices [Feature engineering]</h1>

In [14]:
def offices(df):
    many_offices = []

    for i in range(len(df['osm_offices_points_in_0.001'])):
        if df['osm_offices_points_in_0.001'][i] > 2 or df['osm_offices_points_in_0.005'][i] > 10 or df['osm_offices_points_in_0.0075'][i] > 15 or df['osm_offices_points_in_0.01'][i] > 20:
            many_offices.append(1)
        else:
            many_offices.append(0)

    df['many_offices'] = many_offices

    df.drop(['osm_offices_points_in_0.001', 'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075', 'osm_offices_points_in_0.01'], axis=1, inplace=True)
    
offices(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Food [Feature engineering]</h1>

In [15]:
def food(df):
    many_food = []

    for i in range(len(df['osm_catering_points_in_0.001'])):
        if df['osm_catering_points_in_0.001'][i] > 2 or df['osm_catering_points_in_0.005'][i] > 10 or df['osm_catering_points_in_0.0075'][i] > 15 or df['osm_catering_points_in_0.01'][i] > 20:
            many_food.append(1)
        else:
            many_food.append(0)

    df['many_food'] = many_food

    df.drop(['osm_catering_points_in_0.001', 'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075', 'osm_catering_points_in_0.01'], axis=1, inplace=True)
    
food(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Shops [Feature engineering]</h1>

In [16]:
def shops(df):
    many_shops = []

    for i in range(len(df['osm_shops_points_in_0.001'])):
        if df['osm_shops_points_in_0.001'][i] > 2 or df['osm_shops_points_in_0.005'][i] > 10 or df['osm_shops_points_in_0.0075'][i] > 15 or df['osm_shops_points_in_0.01'][i] > 20:
            many_shops.append(1)
        else:
            many_shops.append(0)

    df['many_shops'] = many_shops

    df.drop(['osm_shops_points_in_0.001', 'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075', 'osm_shops_points_in_0.01'], axis=1, inplace=True)
    
shops(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>>Financial Organizations [Feature engineering]</h1>

In [17]:
def financial_organizations(df):
    many_financial_organizations = []

    for i in range(len(df['osm_finance_points_in_0.001'])):
        if df['osm_finance_points_in_0.001'][i] > 2 or df['osm_finance_points_in_0.005'][i] > 10 or df['osm_finance_points_in_0.0075'][i] > 15 or df['osm_finance_points_in_0.01'][i] > 20:
            many_financial_organizations.append(1)
        else:
            many_financial_organizations.append(0)

    df['many_financial_organizations'] = many_financial_organizations

    df.drop(['osm_finance_points_in_0.001', 'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075', 'osm_finance_points_in_0.01'], axis=1, inplace=True)
    
financial_organizations(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Medicine [Feature engineering]</h1>

In [18]:
def medecine(df):
    many_medicine = []

    for i in range(len(df['osm_healthcare_points_in_0.005'])):
        if df['osm_healthcare_points_in_0.005'][i] > 1 or df['osm_healthcare_points_in_0.0075'][i] > 1 or df['osm_healthcare_points_in_0.01'][i] > 1:
            many_medicine.append(1)
        else:
            many_medicine.append(0)

    df['many_medicine'] = many_medicine

    df.drop(['osm_healthcare_points_in_0.005', 'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01'], axis=1, inplace=True)
    
medecine(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Entertainment [Feature engineering]</h1>

In [19]:
def entertainment(df):
    many_entertainment = []

    for i in range(len(df['osm_leisure_points_in_0.005'])):
        if df['osm_leisure_points_in_0.005'][i] > 1 or df['osm_leisure_points_in_0.0075'][i] > 3 or df['osm_leisure_points_in_0.01'][i] > 1:
            many_entertainment.append(1)
        else:
            many_entertainment.append(0)

    df['many_entertainment'] = many_entertainment

    df.drop(['osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075', 'osm_leisure_points_in_0.01'], axis=1, inplace=True)
    
entertainment(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Historical objects [Feature engineering]</h1>

In [20]:
def hist_obj(df):
    many_historical_objects = []

    for i in range(len(df['osm_historic_points_in_0.005'])):
        if df['osm_historic_points_in_0.005'][i] > 1 or df['osm_historic_points_in_0.0075'][i] > 3 or df['osm_historic_points_in_0.01'][i] > 1:
            many_historical_objects.append(1)
        else:
            many_historical_objects.append(0)

    df['many_historical_objects'] = many_historical_objects

    df.drop(['osm_historic_points_in_0.005', 'osm_historic_points_in_0.0075', 'osm_historic_points_in_0.01'], axis=1, inplace=True)
    
hist_obj(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Buildings [Feature engineering]</h1>

In [21]:
def buildings(df):
    many_buildings = []

    for i in range(len(df['osm_building_points_in_0.001'])):
        if df['osm_building_points_in_0.001'][i] > 2 or df['osm_building_points_in_0.005'][i] > 5:
            many_buildings.append(1)
        else:
            many_buildings.append(0)

    df['many_buildings'] = many_buildings

    df.drop(['osm_building_points_in_0.001', 'osm_building_points_in_0.005'], axis=1, inplace=True)
    
buildings(df_train)

In [22]:
print_unique_values(df_train)

,Feature,Number of values,Values
0,osm_amenity_points_in_0.001,46,"[4, 3, 1, 0, 5, 10, 2, 7, 6, 19, 9, 11, 8, 15,..."
1,osm_amenity_points_in_0.005,412,"[19, 24, 30, 0, 23, 18, 38, 92, 66, 12, 5, 10,..."
2,osm_amenity_points_in_0.0075,804,"[35, 37, 67, 5, 64, 38, 0, 117, 256, 138, 32, ..."
3,osm_amenity_points_in_0.01,1273,"[52, 59, 128, 21, 153, 50, 0, 219, 376, 249, 5..."
4,osm_building_points_in_0.0075,336,"[0, 1, 13, 2, 29, 14, 4, 10, 6, 8, 17, 15, 57,..."
5,osm_building_points_in_0.01,469,"[0, 1, 26, 4, 37, 3, 23, 2, 21, 13, 8, 11, 7, ..."
6,osm_city_nearest_population,170,"[1055397.0, 120184.0, 603961.0, 552105.0, 2320..."
7,osm_crossing_closest_dist,143191,"[0.0277321951377063, 0.0894408118955342, 0.200..."
8,osm_crossing_points_in_0.001,17,"[3, 0, 2, 4, 1, 5, 8, 7, 11, 6, 13, 9, 10, 12,..."
9,osm_crossing_points_in_0.005,108,"[6, 31, 15, 0, 14, 16, 43, 7, 10, 2, 1, 28, 5,..."


<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Hotels [Feature engineering]</h1>

In [23]:
def hotels(df):
    many_hotels = []

    for i in range(len(df['osm_hotels_points_in_0.005'])):
        if df['osm_hotels_points_in_0.005'][i] > 1 or df['osm_hotels_points_in_0.0075'][i] > 1 or df['osm_hotels_points_in_0.01'][i] > 1:
            many_hotels.append(1)
        else:
            many_hotels.append(0)

    df['many_hotels'] = many_hotels

    df.drop(['osm_hotels_points_in_0.005', 'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01'], axis=1, inplace=True)
    
hotels(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Station score [Feature engineering]</h1>

In [24]:
def stations_score(df):
    station_rate = []

    for row in df['osm_train_stop_closest_dist']:
        if row > 4:
            station_rate.append(0)
        elif row <= 4 and row > 1:
            station_rate.append(5)
        else:
            station_rate.append(1)

    df['station_rate'] = station_rate

    df.drop(['osm_train_stop_closest_dist'], axis=1, inplace=True)
    
stations_score(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Stations [Feature engineering]</h1>

In [25]:
def stations(df):
    many_stations = []

    for i in range(len(df['osm_train_stop_points_in_0.005'])):
        if df['osm_train_stop_points_in_0.005'][i] > 1 or df['osm_train_stop_points_in_0.0075'][i] > 1 or df['osm_train_stop_points_in_0.01'][i] > 1:
            many_stations.append(1)
        else:
            many_stations.append(0)

    df['many_stations'] = many_stations

    df.drop(['osm_train_stop_points_in_0.005', 'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01'], axis=1, inplace=True)
    
stations(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Land transport [Feature engineering]</h1>

In [26]:
def land_transport(df):
    many_land_transport = []

    for i in range(len(df['osm_transport_stop_points_in_0.005'])):
        if df['osm_transport_stop_points_in_0.005'][i] > 1 or df['osm_transport_stop_points_in_0.0075'][i] > 1 or df['osm_transport_stop_points_in_0.01'][i] > 1:
            many_land_transport.append(1)
        else:
            many_land_transport.append(0)

    df['many_land_transport'] = many_land_transport

    df.drop(['osm_transport_stop_points_in_0.005', 'osm_transport_stop_points_in_0.0075', 'osm_transport_stop_points_in_0.01',
             'osm_transport_stop_closest_dist', 'osm_subway_closest_dist'], axis=1, inplace=True)
    
land_transport(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Pedestrian crossings [Feature engineering]</h1>

In [27]:
def pedestrian_crossings(df):
    many_pedestrian_crossings = []

    for i in range(len(df['osm_crossing_points_in_0.001'])):
        if df['osm_crossing_points_in_0.001'][i] > 1 or df['osm_crossing_points_in_0.005'][i] > 1 or df['osm_crossing_points_in_0.0075'][i] > 1 or df['osm_crossing_points_in_0.01'][i]:
            many_pedestrian_crossings.append(1)
        else:
            many_pedestrian_crossings.append(0)

    df['many_pedestrian_crossings'] = many_pedestrian_crossings

    df.drop(['osm_crossing_points_in_0.001', 'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075', 'osm_crossing_points_in_0.01', 
            'osm_crossing_closest_dist'], axis=1, inplace=True)
    
pedestrian_crossings(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Culture objects [Feature engineering]</h1>

In [28]:
def culture_objects(df):
    many_culture_objects = []

    for i in range(len(df['osm_culture_points_in_0.001'])):
        if df['osm_culture_points_in_0.001'][i] > 1 or df['osm_culture_points_in_0.005'][i] > 1 or df['osm_culture_points_in_0.0075'][i] > 1  or df['osm_culture_points_in_0.01'][i] > 1:
            many_culture_objects.append(1)
        else:
            many_culture_objects.append(0)

    df['many_culture_objects'] = many_culture_objects

    df.drop(['osm_culture_points_in_0.001', 'osm_culture_points_in_0.005', 'osm_culture_points_in_0.0075', 'osm_culture_points_in_0.01'], axis=1, inplace=True)
    
culture_objects(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Comfort objects [Feature engineering]</h1>

In [29]:
def comfort_objects(df):
    many_comfort_objects = []

    for i in range(len(df['osm_amenity_points_in_0.001'])):
        if df['osm_amenity_points_in_0.001'][i] > 1 or df['osm_amenity_points_in_0.005'][i] > 1 or df['osm_amenity_points_in_0.0075'][i] > 1  or df['osm_amenity_points_in_0.01'][i] > 1:
            many_comfort_objects.append(1)
        else:
            many_comfort_objects.append(0)

    df['many_comfort_objects'] = many_comfort_objects

    df.drop(['osm_amenity_points_in_0.001', 'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075', 'osm_amenity_points_in_0.01'], axis=1, inplace=True)
    
comfort_objects(df_train)

<h1 style='background-color: #defcdc; border: 1px solid #a1d194; padding: 10px; font-weight: 400; text-align:center'>Quick Summary</h1>

In [30]:
print_unique_values(df_train)

,Feature,Number of values,Values
0,osm_building_points_in_0.0075,336,"[0, 1, 13, 2, 29, 14, 4, 10, 6, 8, 17, 15, 57,..."
1,osm_building_points_in_0.01,469,"[0, 1, 26, 4, 37, 3, 23, 2, 21, 13, 8, 11, 7, ..."
2,osm_city_nearest_population,170,"[1055397.0, 120184.0, 603961.0, 552105.0, 2320..."
3,per_square_meter_price,144209,"[139937.5, 60410.71428571428, 45164.7612642905..."
4,reform_count_of_houses_1000,699,"[136, 146, 105, 75, 144, 83, 7, 207, 559, 154,..."
5,reform_count_of_houses_500,275,"[49, 37, 27, 2, 38, 22, 4, 59, 163, 44, 48, 17..."
6,reform_house_population_1000,6207,"[2503.0, 1336.0, 1883.0, 1801.0, 3090.0, 1958...."
7,reform_house_population_500,2367,"[765.0, 514.0, 573.0, 54.0, 619.0, 451.0, 28.0..."
8,reform_mean_floor_count_1000,49018,"[5.762962962962963, 2.8943661971830985, 6.1414..."
9,reform_mean_floor_count_500,17122,"[5.530612244897959, 3.5277777777777777, 7.2222..."


<h1 style='background-color: #fecfffcd; border: 1px solid #d194cf; padding: 10px; font-weight: 400; text-align:center'>Analys</h1>

In [31]:
print_missing_values(df_train)

,dtype,Feature,Number Of Missing Values,Percentage of Missing values
0,float64,osm_city_nearest_population,55,0.02
1,float64,reform_house_population_1000,14596,5.22
2,float64,reform_house_population_500,27234,9.73
3,float64,reform_mean_floor_count_1000,16708,5.97
4,float64,reform_mean_floor_count_500,30168,10.78
5,float64,reform_mean_year_building_1000,16239,5.80
6,float64,reform_mean_year_building_500,29637,10.59


<h1 style='background-color: #fecfffcd; border: 1px solid #d194cf; padding: 10px; font-weight: 400; text-align:center'>Filling NA's</h1>

In [32]:
df_train['reform_house_population_500'] = df_train['reform_house_population_500'].fillna(df_train['reform_house_population_500'].median())
df_train['reform_house_population_1000'] = df_train['reform_house_population_1000'].fillna(df_train['reform_house_population_1000'].median())
df_train['reform_mean_floor_count_1000'] = df_train['reform_mean_floor_count_1000'].fillna(df_train['reform_mean_floor_count_1000'].median())
# df['reform_mean_floor_count_1000'] = df['reform_mean_floor_count_1000'].apply(lambda x: np.log(x+1))
df_train['reform_mean_floor_count_500'] = df_train['reform_mean_floor_count_500'].fillna(df_train['reform_mean_floor_count_500'].median())
df_train['reform_mean_year_building_1000'] = df_train['reform_mean_year_building_1000'].fillna(df_train['reform_mean_year_building_1000'].median())
df_train['reform_mean_year_building_500'] = df_train['reform_mean_year_building_500'].fillna(df_train['reform_mean_year_building_500'].median())

<h1 style='background-color: #fecfffcd; border: 1px solid #d194cf; padding: 10px; font-weight: 400; text-align:center'>'osm_city_nearest_population' missing values = 100%</h1>

In [33]:
print_missing_values(df_train[df_train['osm_city_nearest_population'].isnull()])

,dtype,Feature,Number Of Missing Values,Percentage of Missing values
0,float64,osm_city_nearest_population,55,100.0


#### Summary
All other features contain many missing values, so we will delete all this rows

In [34]:
df_train = df_train[df_train['osm_city_nearest_population'].notna()]

<h1 style='background-color: #fecfffcd; border: 1px solid #d194cf; padding: 10px; font-weight: 400; text-align:center'>'reform_mean_floor_count_500' missing values = 100%</h1>

In [35]:
print_missing_values(df_train[df_train['reform_mean_floor_count_500'].isnull()])

,dtype,Feature,Number Of Missing Values,Percentage of Missing values


<h1 style='background-color: #fecfffcd; border: 1px solid #d194cf; padding: 10px; font-weight: 400; text-align:center'>'reform_mean_year_building_1000' missing values = 100%</h1>

In [36]:
print_missing_values(df_train[df_train['reform_mean_year_building_1000'].isnull()])

,dtype,Feature,Number Of Missing Values,Percentage of Missing values


<h1 style='background-color: #fecfffcd; border: 1px solid #d194cf; padding: 10px; font-weight: 400; text-align:center'>'reform_mean_year_building_500' missing values = 100%</h1>

In [37]:
print_missing_values(df_train[df_train['reform_mean_year_building_500'].isnull()])

,dtype,Feature,Number Of Missing Values,Percentage of Missing values


<h1 style='background-color: #fecfffcd; border: 1px solid #d194cf; padding: 10px; font-weight: 400; text-align:center'>Summary</h1>

In [38]:
print_missing_values(df_train)

,dtype,Feature,Number Of Missing Values,Percentage of Missing values


<h1 style='background-color: #fecfffcd; border: 1px solid #d194cf; padding: 10px; font-weight: 400; text-align:center'>Fill 'reform_mean_floor_count_500' NA's by medians</h1>

In [39]:
print_missing_values(df_train)

,dtype,Feature,Number Of Missing Values,Percentage of Missing values


<h1 style='background-color: #fecfffcd; border: 1px solid #d194cf; padding: 10px; font-weight: 400; text-align:center'>Fill 'floor' NA's</h1>

In [40]:
# df['floor'] = df['floor'].replace({'подвал': -1,'цоколь': 0,'антресоль' : 1.5,
#                                    'фактически на уровне 1 этажа':1,'тех.этаж (6)':6,
#                                     'подвал': 1,'цоколь, 1': 1,
#                                    '1,2,антресоль':1.5,'Подвал':-1,
#                                    'Цоколь':0.6 ,'1,2,3':'1', 'подвал':'1'})


# flors = ['-1','1', '2', '1', '4', '5', '3',
#        '0.6', '10', '6', '1', '1', '1', '1',
#        '5', '1', '12', '15', '13',
#        '1, ', '2', '1', '8', '7',
#        '0', '3', '1',
#        '1', '1', '4', '1,', '9',
#        '1 ', '1', '1', '1',
#        '4', '2', '4', '1', '18', '1',
#        '0', '-1', '1', '3', '1',
#        '1', '3', '1',
#        '1', '3', '-1', '3', '11', '36', '7',
#        '1', '1', '3', '4', '5', '1',
#        '29', '-1, ', '3']
# old_flors = ['подвал', '1', '2', '1.5', '4', '5', '3',
#        '0.6', '10', '6', '1, подвал', '1,2,3,4', '1,2', '1,2,3,4,5',
#        '5, мансарда', '1-й, подвал', '12', '15', '13',
#        '1, подвал, антресоль', 'мезонин', 'подвал, 1-3', '8', '7',
#        '1 (Цокольный этаж)', '3, Мансарда (4 эт)', 'подвал,1',
#        '1, антресоль', '1-3', 'мансарда (4эт)', '1, 2.', '9',
#        'подвал , 1 ', '1, 2', 'подвал, 1,2,3', '1 + подвал (без отделки)',
#        'мансарда', '2,3', '4, 5', '1-й, 2-й', '18', '1',
#        '1, цоколь', 'подвал', '3 (антресоль)', '1, 2, 3',
#        'Цоколь, 1,2(мансарда)', 'подвал, 3. 4 этаж', 'подвал, 1-4 этаж',
#        'подва, 1.2 этаж', '2, 3', '-1', '1.2', '11', '36', '7,8',
#        '1 этаж', '1-й', '3 этаж', '4 этаж', '5 этаж', 'подвал,1,2,3,4,5',
#        '29', 'подвал, цоколь, 1 этаж', '3, мансарда']

# res = dict(zip (old_flors,flors))

# df['floor'] = df['floor'].replace(res)

# df['floor'] = df['floor'].replace({'подвал, 1':'0',np.nan:0})

# df['floor'] = df['floor'].replace({'1 этаж, подвал': '0', np.nan: 0})

# df['floor'].unique()

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Split Data</h1>

In [41]:
X, y = df_train.loc[:, df_train.columns != 'per_square_meter_price'], df_train['per_square_meter_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Linear Regression</h1>

In [42]:
lg = LinearRegression()

lg.fit(X_train, y_train)

mean_squared_error(y_test, lg.predict(X_test))

17311943396.81235

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Decision Tree Regressor</h1>

In [43]:
tree = DecisionTreeRegressor(random_state=3)

tree.fit(X_train, y_train)

mean_squared_error(y_test, tree.predict(X_test))

9961401626.566133

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Random Forest Regressor</h1>

In [44]:
random_tree = RandomForestRegressor(random_state=3)

random_tree.fit(X_train, y_train)

mean_squared_error(y_test, random_tree.predict(X_test))

5895070945.075038

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Gradient Boosting Regressor</h1>

In [45]:
# gb = GradientBoostingRegressor(random_state=3)

# gb.fit(X_train, y_train)

# mean_squared_error(y_test, gb.predict(X_test))

<h1 style='background-color: #dae8fc; border: 1px solid #94add0; padding: 10px; font-weight: 400; text-align:center'>Submission</h1>

In [46]:
df_test = pd.read_csv('../input/raifhackds2021fall/data/test.csv')

In [47]:
ID = df_test['id']
df_test.drop(['id', 'lat', 'lng', 'osm_city_nearest_name', 'date', 'street', 'floor', 'osm_city_closest_dist'], axis=1, inplace=True)

df_test['reform_house_population_500'] = df_test['reform_house_population_500'].fillna(df_test['reform_house_population_500'].median())
df_test['reform_house_population_1000'] = df_test['reform_house_population_1000'].fillna(df_test['reform_house_population_1000'].median())
df_test['reform_mean_floor_count_1000'] = df_test['reform_mean_floor_count_1000'].fillna(df_test['reform_mean_floor_count_1000'].median())
# df['reform_mean_floor_count_1000'] = df['reform_mean_floor_count_1000'].apply(lambda x: np.log(x+1))
df_test['reform_mean_floor_count_500'] = df_test['reform_mean_floor_count_500'].fillna(df_test['reform_mean_floor_count_500'].median())
df_test['reform_mean_year_building_1000'] = df_test['reform_mean_year_building_1000'].fillna(df_test['reform_mean_year_building_1000'].median())
df_test['reform_mean_year_building_500'] = df_test['reform_mean_year_building_500'].fillna(df_test['reform_mean_year_building_500'].median())

In [48]:
location(df_test)
cities_1m(df_test)
make_district(df_test)
offices(df_test)
food(df_test)
shops(df_test)
financial_organizations(df_test)
medecine(df_test)
entertainment(df_test)
hist_obj(df_test)
buildings(df_test)
hotels(df_test)
stations_score(df_test)
stations(df_test)
land_transport(df_test)
pedestrian_crossings(df_test)
culture_objects(df_test)
comfort_objects(df_test)

Moscow: 56
Moscow oblast: 17
Region: 2901
is_million: 1263
is_not_million: 1711


In [49]:
random_tree.predict(df_test)

array([ 50258.27579194,  50742.46750126,  52756.84972143, ...,
        44517.99749628, 100120.44626037,  58398.44838131])

In [50]:
output = pd.DataFrame({'id': ID, 'per_square_meter_price': random_tree.predict(df_test)})
output.to_csv('my_submission.csv', index=False)

print("Your submission was successfully saved!")

Your submission was successfully saved!
